In [7]:
path="C:/Users/cLennon/Documents/GitHub/machine_learning_examples/ann_logistic_extra"
import numpy as np
import pandas as pd
import os
def get_data(path=path):
    df = pd.read_csv(path + '/ecommerce_data.csv')

    # just in case you're curious what's in it
    # df.head()

    # easier to work with numpy array
    data = df.as_matrix()

    X = data[:,:-1]
    Y = data[:,-1]

    # normalize columns 1 and 2
    X[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()
    X[:,2] = (X[:,2] - X[:,2].mean()) / X[:,2].std()

    # create a new matrix X2 with the correct number of columns
    N, D = X.shape
    X2 = np.zeros((N, D+3))
    X2[:,0:(D-1)] = X[:,0:(D-1)] # non-categorical

    # one-hot
    for n in range(N):
        t = int(X[n,D-1])
        X2[n,t+D-1] = 1

    # method 2
    # Z = np.zeros((N, 4))
    # Z[np.arange(N), X[:,D-1].astype(np.int32)] = 1
    # # assign: X2[:,-4:] = Z
    # assert(np.abs(X2[:,-4:] - Z).sum() < 1e-10)

    return X2, Y


def get_binary_data():
    # return only the data from the first 2 classes
    X, Y = get_data()
    X2 = X[Y <= 1]
    Y2 = Y[Y <= 1]
    return X2, Y2


In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle

X,Y=get_data(path)
X,Y=shuffle(X,Y)
Ntrain=int(.7*len(X))
Xtrain,Ytrain=X[:Ntrain],Y[:Ntrain]
Xtest,Ytest=X[Ntrain:],Y[Ntrain:]
model=MLPClassifier(hidden_layer_sizes=(5,5),max_iter=2000)
model.fit(Xtrain,Ytrain)

train_acc=model.score(Xtrain,Ytrain)
test_acc=model.score(Xtest,Ytest)
print("Train accuracy:" +str(train_acc)+" Test accuracy: "+str(test_acc))

Train accuracy:0.968571428571 Test accuracy: 0.92


In [ ]:
import numpy as np
import pandas as pd


def init_weight_and_bias(M1, M2): #input size and output size
    W = np.random.randn(M1, M2) / np.sqrt(M1+M2) #more weights you have, closer to zero they start
    b = np.zeros(M2)
    return W.astype(np.float32), b.astype(np.float32)

def init_filter(shape, poolsz):#used for conv nn
    #Shape is a tuple of 4 values (reason?)and we divide by the sqrt
    # of fan size in times fan size out like with the weights
    w = np.random.randn(*shape) / np.sqrt(np.prod(shape[1:]) + shape[0]*np.prod(shape[2:] / np.prod(poolsz)))
    return w.astype(np.float32)


def relu(x):
    if x>0:
        return x
    return 0
def sigmoid(A):
    return 1 / (1 + np.exp(-A))


def softmax(A):
    expA = np.exp(A)
    return expA / expA.sum(axis=1, keepdims=True)


def sigmoid_cost(T, Y):#calulates cross entropy from definition 
    #for sigmoid binary class
    return -(T*np.log(Y) + (1-T)*np.log(1-Y)).sum()

def cost(T, Y): #cross entropy for more than 2 classes
    return -(T*np.log(Y)).sum()


def cost2(T, Y):
    # same as cost(), just uses the targets to index Y
    # instead of multiplying by a large indicator matrix with mostly 0s
    N = len(T)
    return -np.log(Y[np.arange(N), T]).mean()


def error_rate(targets, predictions):
    return np.mean(targets != predictions)


def y2indicator(y):
    N = len(y)
    K = len(set(y))
    ind = np.zeros((N, K))
    for i in range(N):
        ind[i, y[i]] = 1
    return ind


def getData(path,balance_ones=True):
    # images are 48x48 = 2304 size vectors
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(path+'fer2013.csv'):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0])) #label
            X.append([int(p) for p in row[1].split()])#pixels space sep

    X, Y = np.array(X) / 255.0, np.array(Y) #scale and convert to array

    if balance_ones: #balance classes through repetition
        # balance the 1 class
        X0, Y0 = X[Y!=1, :], Y[Y!=1]
        X1 = X[Y==1, :]
        X1 = np.repeat(X1, 9, axis=0)
        X = np.vstack([X0, X1])
        Y = np.concatenate((Y0, [1]*len(X1)))

    return X, Y


def getImageData(): #r
    X, Y = getData()
    N, D = X.shape
    d = int(np.sqrt(D))
    X = X.reshape(N, 1, d, d) #N samples one color channel, width and height
    return X, Y


def getBinaryData(path):
    Y = []
    X = []
    first = True
    for line in open(path+'fer2013.csv'):
        if first:
            first = False
        else:
            row = line.split(',')
            y = int(row[0])
            if y == 0 or y == 1:
                Y.append(y)
                X.append([int(p) for p in row[1].split()])
    return np.array(X) / 255.0, np.array(Y)


def crossValidation(model, X, Y, K=5):
    # split data into K parts
    X, Y = shuffle(X, Y)
    sz = len(Y) // K
    errors = []
    for k in range(K):
        xtr = np.concatenate([ X[:k*sz, :], X[(k*sz + sz):, :] ])
        ytr = np.concatenate([ Y[:k*sz], Y[(k*sz + sz):] ])
        xte = X[k*sz:(k*sz + sz), :]
        yte = Y[k*sz:(k*sz + sz)]

        model.fit(xtr, ytr)
        err = model.score(xte, yte)
        errors.append(err)
    print("errors:", errors)
    return np.mean(errors)